In [1]:
import gym
import numpy as np
from gymenv.PlaytimeEnv import PlaytimeEnv
from objects.Maneuver import Mission_Maneuver
from objects.Plane import ULM
from function.tools import *
from function.j_methods import * 

env = gym.make('MountainCar-v0')

Cet avion peut voler pendant  52438  s, soit  873  min, soit 2039  km à une vitesse moyenne de  140  km/h et sa vitesse max sera de  200  km/h


In [2]:
# Observation and action space 
obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

The observation space: Box(2,)
The action space: Discrete(3)


In [3]:
env = gym.make('MountainCarContinuous-v0')


In [4]:
import time 
def policy(observation):
    position, velocity = observation
    ret = []
    if velocity < 0:
        ret.append(-0.75)
    elif velocity > 0:
        ret.append(0.75)
    else:
        ret.append(1)
    return ret

num_steps = 150
obs = env.reset()
for step in range(num_steps):
    # take random action, but you can also do something more intelligent
    # action = my_intelligent_agent_fn(obs) 
    action = policy(obs)
    # action = env.action_space.sample()
    # apply the action
    obs, reward, done, info = env.step(action)
    # Render the env
    env.render()

    # Wait a bit before the next frame unless you want to see a crazy fast video
    time.sleep(0.0005)
    
    # If the epsiode is up, then start another one
    if done:
        
        env.reset()

# Close the env
env.close()

In [5]:
param_list = {
    'Plane': [ULM],
    'GoalDistance': [0, 10, 15, 20, 30],
    'RtBDistance': [0, 10, 15, 20, 30],
    'Fuel': [1, 2, 3, 3, 4, 10],
    # Start with just 2 states : good (sunny, no clouds) and bad (cloudy or rainy)
    'Meteo': ["Sunny", "Cloudy", "Misty"],
    'MissionType': [Mission_Maneuver.SCAR],  # , Mission_Maneuver.CAS
    # Add ennemies number afterwards, weaponry
    'Strength': ['Weak', 'Equal', 'Strong'],
    'TimeMin': [0, 300],
}
combinations = get_all_combinations(param_list)
first_c = list(filter(lambda c : c['TimeMin'] == 300 and c['Fuel'] == 3, combinations))[0]
print(first_c)

{'Plane': ULM, 'GoalDistance': 0, 'RtBDistance': 0, 'Fuel': 3, 'Meteo': 'Sunny', 'MissionType': SCAR, 'Strength': 'Weak', 'TimeMin': 300}


In [6]:
env = PlaytimeEnv(first_c)
env.reset()
a = env.action_space.sample()
print(a, env.step(a))

Parameters : c0 :  3
c1 :  300
Min
Target  [3.0, 392.27]
Reset, new obs :  [  1.89 264.58]
OrderedDict([('altitude', 34), ('distance', 2), ('speed', 9)]) ([1.77, 248.54], 0.005656495721709461, False, {})


In [25]:
env = PlaytimeEnv(first_c)

num_steps = 150
obs = env.reset()
for step in range(num_steps):
    # take random action, but you can also do something more intelligent
    # action = my_intelligent_agent_fn(obs) 
    # action = policy(obs)
    action = env.action_choose(obs)
    # apply the action
    obs, reward, done, info = env.step(action)
    print(obs, reward, done, info)
    # Render the env
    # env.render()

    # Wait a bit before the next frame unless you want to see a crazy fast video
    time.sleep(0.001)
    
    # If the episode is up, then start another one
    if done:
        print("Reset now")
        env.reset()
env.close()


Parameters : c0 :  3
c1 :  300
Min
Target  [3.0, 392.27]
Reset, new obs :  [  4.51 592.29]
[4.45, 585.34] 0.003572047301050359 False {}
[4.33, 571.89] 0.004185946438303754 False {}
[4.33, 573.63] 0.0041457857258939135 False {}
[4.21, 559.84] 0.004931946535726772 False {}
[4.09, 547.03] 0.005928089898297691 False {}
[3.97, 534.35] 0.007255967307513697 False {}
[3.91, 528.58] 0.008061779025024564 False {}
[3.85, 521.24] 0.009122048447199299 False {}
[3.73, 507.09] 0.011930526160064712 False {}
[3.61, 494.66] 0.016010784864684848 False {}
[3.61, 494.66] 0.016010784864684848 False {}
[3.61, 494.66] 0.016010784864684848 False {}
[3.61, 495.25] 0.015919054790202777 False {}
[3.55, 489.73] 0.018655672257149786 False {}
[3.55, 489.73] 0.018655672257149786 False {}
[0.43000000000000016, 85.36000000000001]
[3.43, 477.63] 0.027244393103899207 True {}
Reset now
Reset, new obs :  [  2.36 329.44]
[2.3, 322.52] 0.020481310803891442 False {}
[2.31, 320.85] 0.020292290147281455 False {}
[2.31, 320.85] 

# Let's try to add an agent with a deep learning neural network

In [8]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

In [9]:
states = (len(env.observation_space.spaces.keys()),)
actions = len(env.action_space.spaces.keys())  # sum(action.n for action in env.action_space.spaces.values())
num_hidden = 24

In [10]:
print(states, actions)

(2,) 3


In [11]:
del model

NameError: name 'model' is not defined

In [13]:
# Entrée : paramètres de wheel
# Ajouter aussi les choix de plusieurs manoeuvres

inputs = Input(shape=states)
first = Dense(num_hidden, activation='relu')(inputs)
second = Dense(num_hidden, activation='relu')(first)
out_speed = Dense(env.action_space.spaces['speed'].n, activation='linear')(second)
out_altitude = Dense(env.action_space.spaces['altitude'].n, activation='linear')(second)
out_distance = Dense(env.action_space.spaces['distance'].n, activation='linear')(second)

model = Model(inputs=inputs, outputs=[out_speed, out_altitude, out_distance])


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 24)           72          ['input_2[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 24)           600         ['dense_3[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 10)           250         ['dense_4[0][0]']                
                                                                                              

In [15]:
from rl.agents import DDPGAgent, DQNAgent
from rl.policy import BoltzmannQPolicy 
from rl.memory import SequentialMemory

In [18]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [19]:
dqn = build_agent(states, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

AttributeError: 'tuple' object has no attribute 'output'